In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import rcParams
import pickle
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import gc

matplotlib.style.use('ggplot')
rcParams['text.latex.unicode'] = True
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsthm}', r'\usepackage{amsmath}', r'\usepackage{amssymb}', \
                                   r'\usepackage{amsfonts}'
#rcParams['text.latex.preamble']
#rcParams['text.latex.preamble'] = r'\usepackage{amsthm}'

Wmin = 0.005
Wmax = 0.05
Wnsteps = 10
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

Nmin = 5
Nmax = 15
Nnsteps = 10
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps


data = BPV.Data()
data.read_csv("pixel.dist.csv",False)
data = data.data_head(100)
data.order_by_p()
data.df
#Nrange

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,pattern-id,p,plog1onp
0,11000,0.000326,0.002616
1,110000,0.000328,0.002629
2,100000001,0.000342,0.002729
3,111000111,0.000342,0.002732
4,10010000,0.000350,0.002783
5,10010010,0.000351,0.002793
6,1000100,0.000353,0.002806
7,10010,0.000366,0.002897
8,111000,0.000378,0.002977
9,1000001,0.000384,0.003017


In [15]:
#CALCULATE EPSILON-SOLUTIONS

epsilon = 0.2

Hentropies = np.zeros((Wnsteps, Nnsteps))
Hrates = np.zeros((Wnsteps, Nnsteps))
Hcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Htimes = np.zeros((Wnsteps, Nnsteps))
Hindexes = {}


Wentropies = np.zeros((Wnsteps, Nnsteps))
Wrates = np.zeros((Wnsteps, Nnsteps))
Wcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Wtimes = np.zeros((Wnsteps, Nnsteps))
Windexes = {}


it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, nproblems, W, N,i,j))
    
    data.quantize_entropy(N, epsilon)
    
    #Hprbl_exact = BPV.BPV("decgraphH",data,N,W,time_solver=True)
    #Hprbl_exact.solve()
    #Hprbl_exact.pprint_solution()
    
    #Hentropies[i,j] = Hprbl_exact.solution_entropy
    #Hrates[i,j] = Hprbl_exact.solution_rate
    #Hcardinalities[i,j] = Hprbl_exact.solution_cardinality
    #Hindexes[(i,j)] = Hprbl_exact.solution.index
    #Htimes[(i,j)] = Hprbl_exact.solution_time
    
    #Wprbl_exact = BPV.BPV("decgraphW",data,N,W,time_solver=True)
    Wprbl_exact = BPV.BPV("decgraphW",data,N,W,time_solver=True,\
                          use_quantized_entropy=True)
    Wprbl_exact.solve()
    Wprbl_exact.pprint_solution()
    
    Wentropies[i,j] = Wprbl_exact.solution_entropy
    Wrates[i,j] = Wprbl_exact.solution_rate
    Wcardinalities[i,j] = Wprbl_exact.solution_cardinality
    Windexes[(i,j)] = Wprbl_exact.solution.index
    Wtimes[(i,j)] = Wprbl_exact.solution_time
    
    gc.collect()
    it += 1
    



Solving problem 1 out of 100, for W = 0.005 and N = 5 i = 0  j = 0

Solver =  decgraphW 
Entropy =  0.034499234721 
Cardinality =  5 
Rate =  0.00499703910228
Solution time in seconds =  8.858109038992552


Solving problem 2 out of 100, for W = 0.005 and N = 6 i = 0  j = 1

Solver =  decgraphW 
Entropy =  0.0353772207993 
Cardinality =  6 
Rate =  0.00499717265653
Solution time in seconds =  5.326670206995914


Solving problem 3 out of 100, for W = 0.005 and N = 7 i = 0  j = 2

Solver =  decgraphW 
Entropy =  0.0360537568807 
Cardinality =  7 
Rate =  0.0049986551087
Solution time in seconds =  6.134713249994093


Solving problem 4 out of 100, for W = 0.005 and N = 8 i = 0  j = 3

Solver =  decgraphW 
Entropy =  0.0367390497629 
Cardinality =  8 
Rate =  0.00499941636793
Solution time in seconds =  7.649602161007351


Solving problem 5 out of 100, for W = 0.005 and N = 9 i = 0  j = 4

Solver =  decgraphW 
Entropy =  0.0373670720953 
Cardinality =  9 
Rate =  0.00499925610283
Solution

In [20]:
zeroindexes = (Wentropies == 0).nonzero()
totzeros = len(zeroindexes[0])
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
for i,j in zeroindexes:
    print(i,j,totzeros)

In [17]:
#SAVE EPSILON SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot12.entropies",r'wb')
f2 = open("/home/renato/tesi/code/data/plot12.rates",r'wb')
f3 = open("/home/renato/tesi/code/data/plot12.cardinalities",r'wb')
f4= open("/home/renato/tesi/code/data/plot12.indexes",r'wb')

pickle.dump(Wentropies,f1)
pickle.dump(Wrates,f2)
pickle.dump(Wcardinalities,f3)
pickle.dump(Windexes,f4)


In [2]:
#LOAD EPSILON SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot12.entropies",r'rb')
f2 = open("/home/renato/tesi/code/data/plot12.rates",r'rb')
f3 = open("/home/renato/tesi/code/data/plot12.cardinalities",r'rb')
f4 = open("/home/renato/tesi/code/data/plot12.indexes",r'rb')

Wentropies = pickle.load(f1)
Wrates = pickle.load(f2)
Wcardinalities = pickle.load(f3)
Windexes = pickle.load(f4)

In [ ]:
#CALCULATE EXACT SOLUTIONS

Pentropies = np.zeros((Wnsteps, Nnsteps))
Prates = np.zeros((Wnsteps, Nnsteps))
Pcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Ptimes = np.zeros((Wnsteps, Nnsteps))
Pindexes = {}


Pentropies = np.zeros((Wnsteps, Nnsteps))
Prates = np.zeros((Wnsteps, Nnsteps))
Pcardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
Ptimes = np.zeros((Wnsteps, Nnsteps))
Pindexes = {}


it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, nproblems, W, N,i,j))

    Pprbl_exact = BPV.BPV("glpk",data,N,W,time_solver=True)
    Pprbl_exact.solve()
    Pprbl_exact.pprint_solution()
    
    Pentropies[i,j] = Pprbl_exact.solution_entropy
    Prates[i,j] = Pprbl_exact.solution_rate
    Pcardinalities[i,j] = Pprbl_exact.solution_cardinality
    Pindexes[(i,j)] = Pprbl_exact.solution.index
    Ptimes[(i,j)] = Pprbl_exact.solution_time
    
    gc.collect()
    it += 1
    



Solving problem 1 out of 100, for W = 0.005 and N = 5 i = 0  j = 0

Solver = 

In [3]:
# EXACT AND HEUR MEGA PLOT A
fig = plt.figure(figsize=(9,25))
#ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
elev = 22
azimuth = -135
#elev = 40
#azimuth = 47
al = 1
cb_shrink = 0.1
cb_aspect = 10
label_size = 16
title_size = 20
title_rotation = 'horizontal'
title_x = 0.4
title_y = 0
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colormap = cm.gnuplot
_dpi = 80

#gs = gridspec.GridSpec(2,7, width_ratios=[1,0.1,10,0.1,10,0.1,10], height_ratios=[1,1])

gs = gridspec.GridSpec(4,2, width_ratios=[1,100], height_ratios=[1,10,10,10])

title1 = plt.subplot(gs[0,:2])
cex_entropy = plt.subplot(gs[1,0])
ex_entropy = plt.subplot(gs[1,1], projection='3d') #exact entropy

cex_rate = plt.subplot(gs[2,0])
ex_rate = plt.subplot(gs[2,1], projection='3d') #exact rate

cex_card = plt.subplot(gs[3,0])
ex_card = plt.subplot(gs[3,1], projection='3d') #exact cardinality

title1.axis('off')
title1.text(title_x,title_y,"Exact Solution",fontsize=title_size, rotation=title_rotation)

surf_e = ex_entropy.plot_surface(X, Y, Wentropies.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_entropy.set_xlabel('W',fontsize=label_size)
ex_entropy.set_ylabel('N',fontsize=label_size)
ex_entropy.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_entropy.set_zlabel('Entropy',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_entropy.view_init(elev,azimuth)
plt.colorbar(surf_e, cax=cex_entropy)



surf_r = ex_rate.plot_surface(X, Y, Wrates.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_rate.set_xlabel('W',fontsize=label_size)
ex_rate.set_ylabel('N',fontsize=label_size)
ex_rate.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_rate.set_zlabel('Rate',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_rate.view_init(elev,azimuth)
plt.colorbar(surf_r, cax=cex_rate)

surf_c = ex_card.plot_surface(X, Y, Wcardinalities.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_card.set_xlabel('W',fontsize=label_size)
ex_card.set_ylabel('N',fontsize=label_size)
ex_card.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_card.set_zlabel('Cardinality',rotation='vertical',fontsize=label_size, horizontalalignment='right',)
ex_card.view_init(elev,azimuth)
plt.colorbar(surf_c, cax=cex_card)

fig.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/NWsummaryExact.pdf",  bbox_inches='tight')
plt.show()

In [16]:
#CALCULATE GRADIENT, NOT INTERVAL MEASURE, RELATIVE ERROR
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
gx, gy = np.gradient(entropies)
grad = np.zeros((Wnsteps, Nnsteps))
not_interval_measure =  np.zeros((Wnsteps, Nnsteps))
relerr = np.zeros((Wnsteps,Nnsteps))
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    grad[i,j] = np.sqrt(gx[i,j]**2 + gy[i,j]**2)
    not_interval_measure[i,j] = BPV.not_interval_measure(indexes[i,j])
    relerr[i,j] = (entropies[i,j] - heur_entropies[i,j])/entropies[i,j]

In [46]:
#RELATIVE ERRROR
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.6
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = 5*relerr.min()
gradoffset = 0
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

relmin = relerr.min()
relmax = relerr.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, relerr.T, rstride=1, cstride=1, alpha=al, \
                         vmin=relmin, vmax=relmax/2, cmap=colormap)
cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('Relative Error',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)

plt.savefig("/home/renato/tesi/testogit/img/NWrelerr.pdf",  bbox_inches='tight')
plt.show()

In [39]:
#NOT INTERVAL MEASURE
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 5*relerr.min()
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

nimin = not_interval_measure.min()
nimax = not_interval_measure.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, not_interval_measure.T, rstride=1, cstride=1, alpha=al,\
                         vmin=nimin,vmax=nimax, cmap=colormap)
cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$\mathcal{I}(S)$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWnimeasure.pdf",  bbox_inches='tight')
plt.show()

In [28]:
#MEAN VALUES OF INDEXES 1
heur_solmean = np.zeros((Wnsteps,Nnsteps))
ex_solmean = np.zeros((Wnsteps,Nnsteps))

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    hm = (np.array(heur_indexes[i,j])).mean()
    if math.isnan(hm):
        print(i,j, heur_indexes[i,j])
    heur_solmean[i,j] = hm
    ex_solmean[i,j] = (np.array(indexes[i,j])).mean()
    
fig = plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

hmin = 1000
hmax = -2000
for coords,d in np.ndenumerate(heur_solmean):
    if math.isnan(d) == False:
        if d < hmin:
            hmin=d
        elif d> hmax:
            hmax=d
hmin *= 1
hmax *= 1

emin = 1000
emax = -2000
for coords,d in np.ndenumerate(ex_solmean):
    if math.isnan(d) == False:
        if d < emin:
            emin=d
        elif d> emax:
            emax=d
emin *= 1
emax *= 1

X, Y = np.meshgrid(Wrange, Nrange)
elev = 15
azimuth = 124
al = 0.7
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 0.2
ncountourlines = 10
colormap = cm.gnuplot

ax = fig.add_subplot(2, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, ex_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=emin,vmax=emax)
ax.set_title('Exact Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)


grcontourlevels = list(np.arange(hmin,hmax,(hmax-hmin)/ncountourlines ))

ax = fig.add_subplot(2, 1, 2, projection='3d')
surf_r = ax.plot_surface(X, Y, heur_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=hmin,vmax=hmax)
#cset = ax.contour(X, Y, heur_solmean.T, zdir='y', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_title('Heuristic Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWmediumindexes.pdf",  bbox_inches='tight')

plt.show()

In [35]:
#DIFF OF MEAN INDEXES
heur_solmean = np.zeros((Wnsteps,Nnsteps))
ex_solmean = np.zeros((Wnsteps,Nnsteps))

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    hm = (np.array(heur_indexes[i,j])).mean()
    if math.isnan(hm):
        print(i,j, heur_indexes[i,j])
    heur_solmean[i,j] = hm
    ex_solmean[i,j] = (np.array(indexes[i,j])).mean()
    
#fig = plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 24
azimuth = -66
al = 1
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 2
ncountourlines = 10
colormap = cm.gnuplot

fig = plt.figure()
ax = fig.gca(projection='3d')
#diff = ex_solmean.T - heur_solmean.T
diff = heur_solmean.T - ex_solmean.T
dmin = 10
dmax = -20
for coords,d in np.ndenumerate(diff):
    if math.isnan(d) == False:
        if d < dmin:
            dmin=d
        elif d> dmax:
            dmax=d

surf_e = ax.plot_surface(X, Y, diff, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=dmin, vmax=dmax)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}}) - m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)
#plt.savefig("/home/renato/tesi/testo/img/NWdiffmediumindex.pdf",  bbox_inches='tight')
#plt.savefig("/home/renato/tesi/testogit/img/NWdiffmediumindex.pdf")

plt.show()

In [18]:
#TIMES
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = times.min()
#gradlinew = 3
#gradcolor = 'k'
ncountourlines = 7
#zmax = 30
ext3d = False
colormap = cm.gnuplot

#mingr = grad.min()
#maxgr = grad.max()
#grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

tmin = Wtimes.min()
tmax = Wtimes.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, Wtimes.T, rstride=1, cstride=1, alpha=al,\
                         vmin=Wtimes.min(),vmax=Wtimes.max(), cmap=colormap)
#cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
#ax.set_zlim(0,0.5)
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('time',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWtimes.pdf",  bbox_inches='tight')
plt.show()

In [50]:
times

array([[  0.29845797,   0.14410875,   0.18421533,   0.1794002 ,
          0.27965561,   0.14484194,   0.19902669,   0.17123234,
          0.1364557 ,   0.16303439,   0.13594837,   0.14827694,
          0.15935585,   0.18847369,   0.19393904,   0.19392332,
          0.19329098,   0.19078941,   0.19021517,   0.1884725 ,
          0.18634751,   0.18622738,   0.18776808,   0.18858676,
          0.19091554,   0.20689262,   0.20329699,   0.20640492,
          0.189959  ,   0.20818349],
       [  0.1974964 ,   0.17502053,   0.23769195,   0.20096813,
          0.24896523,   0.2202429 ,   0.16808739,   0.19211716,
          0.27103525,   0.19276452,   0.20440865,   0.26146303,
          0.16615335,   0.19527489,   0.1620631 ,   0.21724022,
          0.15721005,   0.16161388,   0.14881849,   0.15204229,
          0.18101164,   0.15456174,   0.18507954,   0.16428643,
          0.1704562 ,   0.20289149,   0.21377267,   0.22927336,
          0.19164727,   0.15361567],
       [  0.16498365,   0.2247